In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count
from pyspark.sql import functions as F


In [2]:
# Criar a sessão Spark
spark = SparkSession.builder.appName("ClientesOptETL").getOrCreate()

In [3]:
# Ler o arquivo JSON
df_clientes_opt = spark.read.json('C:\\Users\\Theuzao\\Desktop\\panvel_datalake\\data\\raw\\clientes_opt.json')


In [4]:
# Converter todas as colunas para String após a leitura
df_clientes_opt = df_clientes_opt.select([F.col(col_name).cast("string").alias(col_name) for col_name in df_clientes_opt.columns])


In [5]:
df_clientes_opt.show()

+------+-------+------+-----+--------------------+
|b_call|b_email|b_push|b_sms|            v_id_cli|
+------+-------+------+-----+--------------------+
|  true|   true|  true| true|010FC87179A2C7940661|
|  null|   true|  null| true|04B2DB249A9623A2371B|
|  true|   true| false| true|0248D7A8AA7D752A2AF9|
| false|  false| false|false|02ABD1DDB59E705FCE49|
|  null|   true|  null| null|050C08C7789F8BF18F10|
|  null|   true|  null| null|044D8241C0BA9EB8A2BD|
| false|  false| false|false|0236AE95865254140BEC|
|  null|   true|  true| null|04E8FC308B3A1CC4CC90|
|  null|   null|  null|false|027B23C4BD585D29DBA9|
| false|  false| false|false|016BC87B3EFE0230E53A|
| false|   true| false|false|01631A90EAF2477D949E|
|  null|   null|  true| null|04889DF12594121513BB|
|  null|   null| false| true|037999CC78E16B39B86D|
|  true|   true|  true| true|006DB98427A7777F6633|
|  true|   true|  true| true|00AB23E7BEC55559643B|
| false|   true| false|false|042510CD68CC179D619D|
|  null|   true|  null|false|00

In [6]:
# Contagem de valores nulos
df_clientes_opt.select([count(when(col(c).isNull(), c)).alias(c) for c in df_clientes_opt.columns]).show()

+------+-------+------+-----+--------+
|b_call|b_email|b_push|b_sms|v_id_cli|
+------+-------+------+-----+--------+
| 43708|  18283| 41144|20099|       0|
+------+-------+------+-----+--------+



In [7]:
# Tranformação dos valores booleanos para Sim, Não, Desconhecido

df_clientes_opt = df_clientes_opt.withColumn(
    "b_call",
    F.when(F.col("b_call").isNull(), "Desconhecido")
     .when(F.col("b_call") == True, "Sim")
     .otherwise("Não")
)

df_clientes_opt = df_clientes_opt.withColumn(
    "b_email",
    F.when(F.col("b_email").isNull(), "Desconhecido")
     .when(F.col("b_email") == True, "Sim")
     .otherwise("Não")
)

df_clientes_opt = df_clientes_opt.withColumn(
    "b_push",
    F.when(F.col("b_push").isNull(), "Desconhecido")
     .when(F.col("b_push") == True, "Sim")
     .otherwise("Não")
)

df_clientes_opt = df_clientes_opt.withColumn(
    "b_sms",
    F.when(F.col("b_sms").isNull(), "Desconhecido")
     .when(F.col("b_sms") == True, "Sim")
     .otherwise("Não")
)

# Mostrar DataFrame após substituição
print("DataFrame após substituição de valores nulos:")
df_clientes_opt.show()

DataFrame após substituição de valores nulos:
+------------+------------+------------+------------+--------------------+
|      b_call|     b_email|      b_push|       b_sms|            v_id_cli|
+------------+------------+------------+------------+--------------------+
|         Sim|         Sim|         Sim|         Sim|010FC87179A2C7940661|
|Desconhecido|         Sim|Desconhecido|         Sim|04B2DB249A9623A2371B|
|         Sim|         Sim|         Não|         Sim|0248D7A8AA7D752A2AF9|
|         Não|         Não|         Não|         Não|02ABD1DDB59E705FCE49|
|Desconhecido|         Sim|Desconhecido|Desconhecido|050C08C7789F8BF18F10|
|Desconhecido|         Sim|Desconhecido|Desconhecido|044D8241C0BA9EB8A2BD|
|         Não|         Não|         Não|         Não|0236AE95865254140BEC|
|Desconhecido|         Sim|         Sim|Desconhecido|04E8FC308B3A1CC4CC90|
|Desconhecido|Desconhecido|Desconhecido|         Não|027B23C4BD585D29DBA9|
|         Não|         Não|         Não|         Não|0

In [8]:
# Salvar o DataFrame em formato CSV
df_clientes_opt.write.mode("overwrite").option("header", "true").csv("C:\\Users\\Theuzao\\Desktop\\panvel_datalake\\data\\transformed\\clientes_opt")

In [9]:
# Encerra a sessão Spark
spark.stop()